In [ ]:
### Testing to understand why some LRP blocks are blank
### If same signs for a row in last layer, this happens, have to depend on a good random seed
### Or fix bias at some high value in the last layer

In [1]:
### Import the required libraries
import numpy as np
import scipy
import matplotlib.pyplot as plt
import cmocean

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter("ignore")

import tensorflow as tf
import tensorflow.keras as keras
from keras import metrics
import innvestigate
from sklearn.model_selection import train_test_split

import os
from os.path import join
import sys

import xarray as xr
import xmitgcm
from xmitgcm import open_mdsdataset
import ecco_v4_py as ecco

import random

# See if GPUs are available
from keras import backend as K
if bool(K._get_available_gpus()):
    print("Running on GPU")
else:
    print("Running on CPU")

# Append to sys.path the absolute path to src/XAIRT
path_list = os.path.abspath('').split('/')
path_src_XAIRT = ''
for link in path_list[:-1]:
    path_src_XAIRT = path_src_XAIRT+link+'/'
sys.path.append(path_src_XAIRT+'/src')

# Now import module XAIRT
from XAIRT import *

### https://stackoverflow.com/questions/36288235/how-to-get-stable-results-with-tensorflow-setting-random-seed ###
### https://keras.io/examples/keras_recipes/reproducibility_recipes/ ###
SEED = 42
keras.utils.set_random_seed(SEED)
tf.config.experimental.enable_op_determinism()

tf.compat.v1.disable_eager_execution()

Running on GPU


2024-04-17 19:53:57.634258: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-17 19:53:59.111287: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38221 MB memory:  -> device: 0, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:21:00.0, compute capability: 8.0
2024-04-17 19:53:59.113596: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 38221 MB memory:  -> device: 1, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:81:00.0, compute capability: 8.0
2024-04-17 19:53:59.115894: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/rep

In [2]:
### Sverdrup 
# mainDir_r4  = '/scratch2/pillarh/eccov4r4'
# mainDir_r5  = '/scratch2/pillarh/eccov4r5'
# gridDir  = mainDir_r4 + '/GRID'
# thetaDir = mainDir_r5 + '/V4r5/diags_daily/SST_day_mean'
# thetaDir_ext = mainDir_r5 + '/V4r5_ext_2020_2023_Jun/diags_daily/SST_day_mean'

## LS6
mainDir_r4 = '/work/07665/shrey911/ls6/LRP_eccov4r4_data'
mainDir_r5 = '/work/07665/shrey911/ls6/LRP_eccov4r5_data'
gridDir  = mainDir_r5 + '/GRID'
thetaDir = mainDir_r5 + '/SST_day_mean'
thetaDir_ext = mainDir_r5 + '/SST_day_mean_ext_2020_2023_Jun'

# For Sverdrup
# ds_r4 = xr.open_dataset(f'/scratch2/shreyas/LRP_eccov4r4_data/thetaSurfECCOv4r4.nc')
# For LS6
ds_r4 = xr.open_dataset(mainDir_r4 + '/thetaSurfECCOv4r4.nc')

# SSH has to be kept because someone used the SSH metadata for SST, 
# It's not a bug in this code but a hack to handle an existing bug.
temp = xmitgcm.open_mdsdataset(data_dir = thetaDir,
                             grid_dir = gridDir,
                             extra_variables = dict(SSH = dict(dims=['k','j','i'],
                                                               attrs = dict(standard_name="SST",
                                                                            long_name="Sea Surface Temperature",
                                                                            units="degC"))))
temp["SST"] = temp["SSH"]
temp = temp.drop(["SSH"])
    
temp_ext = xmitgcm.open_mdsdataset(data_dir = thetaDir_ext,
                                 grid_dir = gridDir,
                                 extra_variables = dict(SST = dict(dims=['k','j','i'],
                                                                   attrs = dict(standard_name="SST",
                                                                                long_name="Sea Surface Temperature",
                                                                                units="degC"))))

ds_r5 = xr.concat([temp, temp_ext], "time")

In [3]:
ds_SST = xr.open_dataset(mainDir_r5+'/SST_all.nc')
SST = ds_SST['SST'].data

In [4]:
hFacC = ecco.read_llc_to_tiles(gridDir, 'hFacC.data')
hFacC_mask = hFacC > 0
hFacC_mask = hFacC_mask.astype(float)

XC = ds_r4['XC'].data
YC = ds_r4['YC'].data

latMask = YC > -20.0
latMask = latMask.astype(float)

maskFinal = hFacC_mask * latMask
NaNmaskFinal = np.copy(maskFinal)
NaNmaskFinal[NaNmaskFinal == 0] = np.nan

da_XC = xr.DataArray(
    data=ds_r4['XC'].data,
    dims=["tile", "j", "i"],
    coords=dict(
        tile = ds_r4['tile'].data,
        j    = ds_r4['j'].data,
        i    = ds_r4['i'].data,
    ),
    attrs=dict(description="XC"),
)

da_YC = xr.DataArray(
    data=ds_r4['YC'].data,
    dims=["tile", "j", "i"],
    coords=dict(
        tile = ds_r4['tile'].data,
        j    = ds_r4['j'].data,
        i    = ds_r4['i'].data,
    ),
    attrs=dict(description="YC"),
)

da_hFacC_mask = xr.DataArray(
    data=hFacC_mask,
    dims=["tile", "j", "i"],
    coords=dict(
        tile = ds_r4['tile'].data,
        j    = ds_r4['j'].data,
        i    = ds_r4['i'].data,
    ),
    attrs=dict(description="hFacC mask 2D 1 if > 0, else 0"),
)

da_latMask = xr.DataArray(
    data=latMask,
    dims=["tile", "j", "i"],
    coords=dict(
        tile = ds_r4['tile'].data,
        j    = ds_r4['j'].data,
        i    = ds_r4['i'].data,
    ),
    attrs=dict(description="Latitude Mask 1 if > -20, else 0"),
)

da_maskFinal = xr.DataArray(
    data=maskFinal,
    dims=["tile", "j", "i"],
    coords=dict(
        tile = ds_r4['tile'].data,
        j    = ds_r4['j'].data,
        i    = ds_r4['i'].data,
    ),
    attrs=dict(description="Mask 2D 1 if > 0, else 0"),
)

da_NaNmaskFinal = xr.DataArray(
    data=NaNmaskFinal,
    dims=["tile", "j", "i"],
    coords=dict(
        tile = ds_r4['tile'].data,
        j    = ds_r4['j'].data,
        i    = ds_r4['i'].data,
    ),
    attrs=dict(description="Mask 2D True if > 0, else NaN"),
)

wetpoints = np.nonzero(maskFinal.data)
da_wetpoints = xr.DataArray(
    data=np.asarray(wetpoints),
    dims=["wetpoints_dim", "num_wetpoints"],
    coords=dict(
        wetpoints_dim = np.arange(np.asarray(wetpoints).shape[0], dtype = int),
        num_wetpoints = np.arange(np.asarray(wetpoints).shape[1], dtype = int),
    ),
    attrs=dict(description="indices of wetpoints in the order (tile, j, i) in the three rows"),
)

da_SST = xr.DataArray(
    data=SST,
    dims=["time", "tile", "j", "i"],
    coords=dict(
        time = ds_r5['time'].data[:SST.shape[0]],
        tile = ds_r4['tile'].data,
        j    = ds_r4['j'].data,
        i    = ds_r4['i'].data,
    ),
    attrs=dict(description="SST field in llc format"),
)

ds = xr.Dataset()
ds = ds.assign(XC           = da_XC,
               YC           = da_YC,
               hFacC_mask   = da_hFacC_mask,
               latMask      = da_latMask,
               maskFinal    = da_maskFinal,
               NaNmaskFinal = da_NaNmaskFinal,
               wetpoints    = da_wetpoints,
               SST          = da_SST)

load_binary_array: loading file /work/07665/shrey911/ls6/LRP_eccov4r5_data/GRID/hFacC.data
load_binary_array: data array shape  (1170, 90)
load_binary_array: data array type  >f4
llc_compact_to_faces: dims, llc  (1170, 90) 90
llc_compact_to_faces: data_compact array type  >f4
llc_faces_to_tiles: data_tiles shape  (13, 90, 90)
llc_faces_to_tiles: data_tiles dtype  >f4


In [5]:
def anomalize_new(field, num_years = 31, first_leap_year_idx = 0):
    
    leap_yr_offsets_jan_feb   = np.array([0,1,1,1,1,2,2,2,2,3,3,3,3,4,4,4,4,5,5,5,5,6,6,6,6,7,7,7,7,8,8])
    leap_yr_offsets_after_feb = np.array([1,1,1,1,2,2,2,2,3,3,3,3,4,4,4,4,5,5,5,5,6,6,6,6,7,7,7,7,8,8,8])

    if len(field.shape) > 1:
        seasonal_trend = np.zeros((366, field.shape[1]))
    else:
        seasonal_trend = np.zeros((366,))

    #### Calculate seasonal trend
    
    # Jan 1 - Feb 28
    for d in range(59):
        same_cal_days_idx=[d+365*year+leap_yr_offsets_jan_feb[year] for year in range(num_years)]
        # Remove mean
        field[same_cal_days_idx] = scipy.signal.detrend(field[same_cal_days_idx], 
                                                        axis=0, 
                                                        type='constant', 
                                                        overwrite_data=False)
        # Remove linear trend
        field[same_cal_days_idx] = scipy.signal.detrend(field[same_cal_days_idx], 
                                                        axis=0, 
                                                        type='linear', 
                                                        overwrite_data=False)
    
    # Feb 29 starting 1996, so year 2 in 0-indexing
    same_cal_days_idx=[365*year+59+int(year/4) for year in range(first_leap_year_idx,num_years,4)]
    # Remove mean
    field[same_cal_days_idx] = scipy.signal.detrend(field[same_cal_days_idx], 
                                                    axis=0, 
                                                    type='constant', 
                                                    overwrite_data=False)
    # Remove linear trend
    field[same_cal_days_idx] = scipy.signal.detrend(field[same_cal_days_idx], 
                                                    axis=0, 
                                                    type='linear', 
                                                    overwrite_data=False)
            
    # Mar 1 - Dec 31
    for d in range(60,366):
        same_cal_days_idx=[d-1+365*year+leap_yr_offsets_after_feb[year] for year in range(num_years)]
        # Remove mean
        field[same_cal_days_idx] = scipy.signal.detrend(field[same_cal_days_idx], 
                                                        axis=0, 
                                                        type='constant', 
                                                        overwrite_data=False)
        # Remove linear trend
        field[same_cal_days_idx] = scipy.signal.detrend(field[same_cal_days_idx], 
                                                        axis=0, 
                                                        type='linear', 
                                                        overwrite_data=False)

    return field

In [6]:
X = ds['SST'].data[:,wetpoints[0],wetpoints[1],wetpoints[2]].copy()
X = anomalize_new(X)
X_full = X.copy()
X = X[30:-30]

y = ds['SST'].isel(tile = 10, j = 1, i = 43).data.copy()
y = anomalize_new(y)
# https://stackoverflow.com/questions/13728392/moving-average-or-running-mean
y = np.convolve(y, np.ones(61)/61, mode='valid')
oneHotCost = np.zeros((y.shape[0], 2), dtype = int)
oneHotCost[:,0] = y >= 0.0
oneHotCost[:,1] = y <  0.0

In [7]:
da_X = xr.DataArray(
    data=X,
    dims=["time_allData", "num_wetpoints"],
    coords=dict(
        time_allData  = ds['time'].data[30:-30],
        num_wetpoints = ds['num_wetpoints'].data,
    ),
    attrs=dict(description="All data as matrix X; Mean removed and delinearized for each calendar day."),
)

da_y = xr.DataArray(
    data=y,
    dims=["time_allData"],
    coords=dict(
        time_allData  = ds['time'].data[30:-30],
    ),
    attrs=dict(description="All cost function y; Mean removed and delinearized for each calendar day."),
)

da_X_full = xr.DataArray(
    data=X_full,
    dims=["time", "num_wetpoints"],
    coords=dict(
        time          = ds['time'],
        num_wetpoints = ds['num_wetpoints'].data,
    ),
    attrs=dict(description="All data without accounting for conv filter as matrix X_full; Mean removed and delinearized for each calendar day."),
)

da_oneHotCost = xr.DataArray(
    data=oneHotCost,
    dims=["time_allData", "NN_output_dim"],
    coords=dict(
        time_allData  = ds['time'].data[30:-30],
        NN_output_dim = np.array([0,1]),
    ),
    attrs=dict(description="All cost function as one-hot vector."),
)

ds = ds.assign(X          = da_X,
               y          = da_y,
               X_full     = da_X_full,
               oneHotCost = da_oneHotCost)

## Classification

In [8]:
def LRP(model, model_wo_softmax, X, y_true, lagSteps, lrp_methods, 
        suffix, normalizeDict, **kwargs):

    result = {}
    cost_NN = model.predict(X)
    result[f'score_{suffix}'] = model.evaluate(X, y_true, verbose=0)

    pred_NN = cost_NN.copy()
    pred_NN[:,0] = pred_NN[:,0] > 0.5
    pred_NN[:,1] = pred_NN[:,1] > 0.5

    idx_NN_pos = []
    idx_NN_neg = []

    if lagSteps >= 0:
        for i in range(len(y_true[lagSteps:,0])):
            if y_true[lagSteps+i,0] == 1 and pred_NN[i,0] == 1:
                idx_NN_pos.append(i)
            if y_true[lagSteps+i,1] == 1 and pred_NN[i,1] == 1:
                idx_NN_neg.append(i)
    else:
        for i in range(len(y_true[:lagSteps,0])):
            if y_true[i,0] == 1 and pred_NN[i-lagSteps,0] == 1:
                idx_NN_pos.append(i-lagSteps)
            if y_true[i,1] == 1 and pred_NN[i-lagSteps,1] == 1:
                idx_NN_neg.append(i-lagSteps)

    result[f'idx_NN_pos'] = idx_NN_pos
    result[f'idx_NN_neg'] = idx_NN_neg

    rel = np.zeros((len(idx_NN_pos), 13, 90, 90))
    rel[:,:,:,:] = np.nan
    rel[:,wetpoints[0],wetpoints[1],wetpoints[2]] = X[idx_NN_pos]
    result[f'samples_correct_pos_{suffix}'] = rel

    rel = np.zeros((len(idx_NN_neg), 13, 90, 90))
    rel[:,:,:,:] = np.nan
    rel[:,wetpoints[0],wetpoints[1],wetpoints[2]] = X[idx_NN_neg]
    result[f'samples_correct_neg_{suffix}'] = rel
    
    for method in lrp_methods:

        title = method['title']
        
        print(f'Analyze using {title} for {suffix} data')
        
        Xplain = XAIR(model_wo_softmax, method, 'classic', X[idx_NN_pos[:10]], 
                      normalizeDict, **kwargs)
        a, _  = Xplain.quick_analyze()
        
        rel = np.zeros((a.shape[0], 13, 90, 90))
        rel[:,:,:,:] = np.nan
        rel[:,wetpoints[0],wetpoints[1],wetpoints[2]] = a
        result[method['title']+f'_pos_{suffix}'] = rel

        Xplain = XAIR(model_wo_softmax, method, 'classic', X[idx_NN_neg[:10]], 
                      normalizeDict, **kwargs)
        a, _  = Xplain.quick_analyze()
        
        rel = np.zeros((a.shape[0], 13, 90, 90))
        rel[:,:,:,:] = np.nan
        rel[:,wetpoints[0],wetpoints[1],wetpoints[2]] = a
        result[method['title']+f'_neg_{suffix}'] = rel
        
    return result

def quickSetup(X, y,
               test_split_frac,
               val_split_frac,
               lrp_methods,
               lagSteps,
               decay_func = None,
               **NNkwargs):
    
    result = {}

    idx = int(X.shape[0]*(1-test_split_frac))
    X_train = X[:idx]
    oneHotCost_train = oneHotCost[:idx]
    X_test = X[idx:]
    oneHotCost_test = oneHotCost[idx:]

    keras.backend.clear_session()
    sgd = keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
    NNkwargs['optimizer'] = sgd

    # Split the data into train and validation sets
    if lagSteps > 0:
        x_t, x_v, oneHotCost_t, oneHotCost_v = train_test_split(X_train[:-lagSteps], oneHotCost_train[lagSteps:], 
                                                                test_size=val_split_frac, shuffle= True, random_state=42)
    elif lagSteps == 0:
        x_t, x_v, oneHotCost_t, oneHotCost_v = train_test_split(X_train, oneHotCost_train, 
                                                                test_size=val_split_frac, shuffle= True, random_state=42)
    else:
        x_t, x_v, oneHotCost_t, oneHotCost_v = train_test_split(X_train[-lagSteps:], oneHotCost_train[:lagSteps], 
                                                                test_size=val_split_frac, shuffle= True, random_state=42)

    K = TrainFullyConnectedNN(x_t, oneHotCost_t, validation_data = (x_v, oneHotCost_v), **NNkwargs)
                   
    best_model = K.quickTrain(decay_func)
    best_model_wo_softmax = innvestigate.model_wo_softmax(best_model)
                   
    result['QoI_predict'] = best_model.predict(X)
    result['QoI_predict_train'] = best_model.predict(X_train)
    result['QoI_predict_test'] = best_model.predict(X_test)
                   
    normalizeDict = {'bool_': True, 'kind': 'MaxAbs'}
    kwargs = {'y_ref': 0.00}

    # LRP for all data
    result_train = LRP(best_model, best_model_wo_softmax, 
                       X, oneHotCost, lagSteps, lrp_methods, 
                       suffix = 'all', normalizeDict = normalizeDict, **kwargs)
    result.update(result_train)
    
    # # LRP for training data
    # result_train = LRP(best_model, best_model_wo_softmax, 
    #                    X_train, oneHotCost_train, lagSteps, lrp_methods, 
    #                    suffix = 'train', normalizeDict = normalizeDict, **kwargs)
    # result.update(result_train)

    # # LRP for test data
    # result_test = LRP(best_model, best_model_wo_softmax, 
    #                    X_test, oneHotCost_test, lagSteps, lrp_methods, 
    #                    suffix = 'test', normalizeDict = normalizeDict, **kwargs)
    # result.update(result_test)
        
    return result

In [9]:
Layers = [{'size': X.shape[1], 'activation': None     , 'use_bias': None},
          {'size': 8         , 'activation': 'relu'   , 'use_bias': True, 
           'l1_w_reg': 0.0, 'l1_b_reg': 0.0, 'l2_w_reg': 10.0, 'l2_b_reg': 10.0},
          {'size': 8         , 'activation': 'relu'   , 'use_bias': True, 
           'l1_w_reg': 0.0, 'l1_b_reg': 0.0, 'l2_w_reg': 0.01, 'l2_b_reg': 0.01},
          {'size': 2         , 'activation': 'softmax', 'use_bias': True, 
           'l1_w_reg': 0.0, 'l1_b_reg': 0.0, 'l2_w_reg': 0.01, 'l2_b_reg': 0.01}]

Losses = [{'kind': 'categorical_crossentropy', 'weight': 1.0}]

LRPDict_theta = {}
lagStepsList = [-60,-30,0]#,30,60,90,120,150,180]

# learning rate schedule
def step_decay(epoch):
    initial_lrate = 0.01
    drop = 0.70710678118 # 0.5^0.5
    epochs_drop = 50
    lrate = initial_lrate * drop**np.floor((1+epoch)/epochs_drop)
    return lrate

methods = [# dict(name='lrp.alpha_1_beta_0', title = 'LRP-A1B0', optParams = {}),
           # dict(name='lrp.alpha_1_beta_0', title = 'LRP-A1B0-W2', optParams = {'input_layer_rule':'WSquare'}),
           dict(name='lrp.alpha_1_beta_0', title = 'LRP-A1B0-B' , optParams = {'input_layer_rule':'Bounded'})]

In [10]:
weight_for_pos = len(oneHotCost[:-2161,0]) / np.sum(oneHotCost[:-2161,0])
weight_for_neg = len(oneHotCost[:-2161,1]) / np.sum(oneHotCost[:-2161,1])
class_weight = {0: weight_for_pos, 1: weight_for_neg}
class_weight

{0: 2.18692936088419, 1: 1.84251012145749}

In [11]:
for i in range(len(lagStepsList)):

    print(f'Lag: {lagStepsList[i]} days, for Theta')
    
    NNkwargs = {'losses': Losses,
                'metrics': [metricF1, # Custom f1 since metrics.F1Score(name='f1') is not available before tf v2.13
                            'accuracy',
                            'categorical_crossentropy',
                            # metrics.CategoricalCrossentropy(name='cross_entropy'),  # (model's loss-L2 reg loss)
                            # metrics.MeanSquaredError(name='brier_score'),
                            # metrics.TruePositives(name='tp'),
                            # metrics.FalsePositives(name='fp'),
                            # metrics.TrueNegatives(name='tn'),
                            # metrics.FalseNegatives(name='fn'), 
                            # metrics.BinaryAccuracy(name='accuracy'),
                            # metrics.Precision(name='precision'),
                            # metrics.Recall(name='recall'),
                            # metrics.AUC(name='auc'),
                            # metrics.AUC(name='prc', curve='PR'), # precision-recall curve],
                           ],
                'batch_size': 128, 'epochs': 1000, #'validation_split': 0.1,
                'filename': f'model{lagStepsList[i]}_noL1', 'dirname': os.path.abspath(''),
                'random_nn_seed': 42, 'class_weight': class_weight,
                'custom_objects': {'metricF1': metricF1}, 'verbose': 2}
    
    LRPDict_theta[f'LRP{lagStepsList[i]}'] = quickSetup(X, y, test_split_frac = 2161.0/11263.0,
                                                        val_split_frac = 0.2,
                                                        lrp_methods = methods,
                                                        lagSteps = lagStepsList[i],
                                                        decay_func = step_decay,
                                                        layers = Layers, **NNkwargs)

Lag: -60 days, for Theta
Train on 7233 samples, validate on 1809 samples


2024-04-17 19:56:21.582674: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38221 MB memory:  -> device: 0, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:21:00.0, compute capability: 8.0
2024-04-17 19:56:21.584728: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 38221 MB memory:  -> device: 1, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:81:00.0, compute capability: 8.0
2024-04-17 19:56:21.586770: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 38221 MB memory:  -> device: 2, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:e2:00.0, compute capability: 8.0
2024-04-17 19:56:21.603915: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled


Epoch 1/1000


2024-04-17 19:56:22.509878: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.



Epoch 1: val_loss improved from inf to 1.26326, saving model to /home1/07665/shrey911/XAIRT/examples_TomsQoI/model-60_noL1.h5
7233/7233 - 1s - loss: 9.6893 - metricF1: 0.7133 - accuracy: 0.7124 - categorical_crossentropy: 0.8252 - val_loss: 1.2633 - val_metricF1: 0.6791 - val_accuracy: 0.6810 - val_categorical_crossentropy: 0.5803 - lr: 0.0100 - 1s/epoch - 160us/sample
Epoch 2/1000

Epoch 2: val_loss did not improve from 1.26326
7233/7233 - 0s - loss: 1.6890 - metricF1: 0.7490 - accuracy: 0.7493 - categorical_crossentropy: 0.5612 - val_loss: 1.3771 - val_metricF1: 0.7389 - val_accuracy: 0.7518 - val_categorical_crossentropy: 0.5139 - lr: 0.0100 - 362ms/epoch - 50us/sample
Epoch 3/1000

Epoch 3: val_loss improved from 1.26326 to 1.11534, saving model to /home1/07665/shrey911/XAIRT/examples_TomsQoI/model-60_noL1.h5
7233/7233 - 1s - loss: 1.7008 - metricF1: 0.7287 - accuracy: 0.7287 - categorical_crossentropy: 0.5665 - val_loss: 1.1153 - val_metricF1: 0.6556 - val_accuracy: 0.6562 - val_

2024-04-17 20:02:40.326213: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38221 MB memory:  -> device: 0, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:21:00.0, compute capability: 8.0
2024-04-17 20:02:40.328264: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 38221 MB memory:  -> device: 1, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:81:00.0, compute capability: 8.0
2024-04-17 20:02:40.330297: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 38221 MB memory:  -> device: 2, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:e2:00.0, compute capability: 8.0



Epoch 1: val_loss improved from inf to 0.99604, saving model to /home1/07665/shrey911/XAIRT/examples_TomsQoI/model-30_noL1.h5
7257/7257 - 1s - loss: 10.1896 - metricF1: 0.6985 - accuracy: 0.6985 - categorical_crossentropy: 1.0232 - val_loss: 0.9960 - val_metricF1: 0.7070 - val_accuracy: 0.7102 - val_categorical_crossentropy: 0.5818 - lr: 0.0100 - 666ms/epoch - 92us/sample
Epoch 2/1000

Epoch 2: val_loss did not improve from 0.99604
7257/7257 - 0s - loss: 1.6243 - metricF1: 0.7508 - accuracy: 0.7511 - categorical_crossentropy: 0.5374 - val_loss: 1.7494 - val_metricF1: 0.4752 - val_accuracy: 0.4700 - val_categorical_crossentropy: 1.0084 - lr: 0.0100 - 400ms/epoch - 55us/sample
Epoch 3/1000

Epoch 3: val_loss did not improve from 0.99604
7257/7257 - 0s - loss: 1.6395 - metricF1: 0.7526 - accuracy: 0.7527 - categorical_crossentropy: 0.5221 - val_loss: 1.0399 - val_metricF1: 0.7371 - val_accuracy: 0.7344 - val_categorical_crossentropy: 0.5337 - lr: 0.0100 - 389ms/epoch - 54us/sample
Epoch 

2024-04-17 20:09:04.545130: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38221 MB memory:  -> device: 0, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:21:00.0, compute capability: 8.0
2024-04-17 20:09:04.547205: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 38221 MB memory:  -> device: 1, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:81:00.0, compute capability: 8.0
2024-04-17 20:09:04.549242: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 38221 MB memory:  -> device: 2, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:e2:00.0, compute capability: 8.0



Epoch 1: val_loss improved from inf to 1.33812, saving model to /home1/07665/shrey911/XAIRT/examples_TomsQoI/model0_noL1.h5
7281/7281 - 1s - loss: 10.3309 - metricF1: 0.6078 - accuracy: 0.6079 - categorical_crossentropy: 1.1184 - val_loss: 1.3381 - val_metricF1: 0.6972 - val_accuracy: 0.6958 - val_categorical_crossentropy: 0.7446 - lr: 0.0100 - 623ms/epoch - 86us/sample
Epoch 2/1000

Epoch 2: val_loss improved from 1.33812 to 0.97407, saving model to /home1/07665/shrey911/XAIRT/examples_TomsQoI/model0_noL1.h5
7281/7281 - 0s - loss: 1.5754 - metricF1: 0.7497 - accuracy: 0.7495 - categorical_crossentropy: 0.5472 - val_loss: 0.9741 - val_metricF1: 0.7889 - val_accuracy: 0.7831 - val_categorical_crossentropy: 0.5010 - lr: 0.0100 - 408ms/epoch - 56us/sample
Epoch 3/1000

Epoch 3: val_loss did not improve from 0.97407
7281/7281 - 0s - loss: 1.5970 - metricF1: 0.7715 - accuracy: 0.7715 - categorical_crossentropy: 0.5165 - val_loss: 1.2119 - val_metricF1: 0.6542 - val_accuracy: 0.6579 - val_c

In [13]:
import innvestigate
import innvestigate.utils as iutils
from innvestigate.analyzer.base import AnalyzerBase

model = keras.models.load_model('model0_noL1.h5', custom_objects={'metricF1': metricF1})
model_wo_softmax = innvestigate.model_wo_softmax(model)
Analyze = innvestigate.create_analyzer('lrp.alpha_1_beta_0', model_wo_softmax, input_layer_rule='Bounded')
sample = LRPDict_theta['LRP0']['samples_correct_neg_all'][0,wetpoints[0],wetpoints[1],wetpoints[2]][np.newaxis,:]

a = Analyze.analyze(sample)

In [15]:
model_wo_softmax.predict(LRPDict_theta['LRP0']['samples_correct_neg_all'][:,wetpoints[0],wetpoints[1],wetpoints[2]])

array([[-0.11217165,  0.11217093],
       [-0.19007564,  0.19007492],
       [-0.36101353,  0.36101282],
       ...,
       [-0.8978359 ,  0.8978352 ],
       [-0.9064645 ,  0.9064638 ],
       [-0.8885796 ,  0.8885789 ]], dtype=float32)

In [39]:
Analyze = innvestigate.create_analyzer('lrp.alpha_1_beta_0', model_wo_softmax, input_layer_rule='Bounded')
sample = LRPDict_theta['LRP0']['samples_correct_neg_all'][0,wetpoints[0],wetpoints[1],wetpoints[2]][np.newaxis,:]

a = Analyze.analyze(sample)
np.sum(a)

0.0

In [42]:
Analyze = innvestigate.create_analyzer('lrp.z', model_wo_softmax)
sample = LRPDict_theta['LRP0']['samples_correct_neg_all'][0,wetpoints[0],wetpoints[1],wetpoints[2]][np.newaxis,:]

a = Analyze.analyze(sample)
np.sum(a)

-0.90291214

In [16]:
model_wo_softmax.predict(LRPDict_theta['LRP0']['samples_correct_pos_all'][:,wetpoints[0],wetpoints[1],wetpoints[2]])

array([[ 1.1364136 , -1.1364143 ],
       [ 1.0987637 , -1.0987644 ],
       [ 1.0798368 , -1.0798376 ],
       ...,
       [ 0.32334507, -0.32334578],
       [ 0.16875911, -0.16875982],
       [ 0.06148565, -0.06148636]], dtype=float32)

In [40]:
Analyze = innvestigate.create_analyzer('lrp.alpha_1_beta_0', model_wo_softmax, input_layer_rule='Bounded')
sample = LRPDict_theta['LRP0']['samples_correct_pos_all'][0,wetpoints[0],wetpoints[1],wetpoints[2]][np.newaxis,:]

a = Analyze.analyze(sample)
np.sum(a)

2.1582303

In [41]:
Analyze = innvestigate.create_analyzer('lrp.z', model_wo_softmax)
sample = LRPDict_theta['LRP0']['samples_correct_pos_all'][0,wetpoints[0],wetpoints[1],wetpoints[2]][np.newaxis,:]

a = Analyze.analyze(sample)
np.sum(a)

2.1518795

In [47]:
model_wo_softmax.layers[-1].get_weights()[0], model_wo_softmax.layers[-1].get_weights()[1]

(array([[ 6.0071580e-02, -6.0071431e-02],
        [ 9.2835146e-01, -9.2834914e-01],
        [ 2.4040677e-01, -2.4040617e-01],
        [ 7.8466892e-02, -7.8466676e-02],
        [ 1.2473488e+00, -1.2473454e+00],
        [ 1.7257469e-02, -1.7257426e-02],
        [ 4.8565216e-06, -4.8533316e-06],
        [ 6.7566639e-01, -6.7566454e-01]], dtype=float32),
 array([-1.4823556,  1.4823549], dtype=float32))

In [48]:
model_wo_softmax.layers[-2].get_weights()[0], model_wo_softmax.layers[-2].get_weights()[1]

(array([[-2.4080284e-02, -3.6989418e-01, -9.6345730e-02, -3.1426691e-02,
         -4.8739284e-01, -6.9225379e-03, -1.9367822e-06, -2.7128395e-01],
        [ 3.2047793e-02,  4.9601376e-01,  1.2823777e-01,  4.1892041e-02,
          6.7177314e-01,  9.1995047e-03,  2.5734919e-06,  3.5997611e-01],
        [ 3.7707929e-02,  5.8189601e-01,  1.5090019e-01,  4.9164824e-02,
          7.8189421e-01,  1.0816029e-02,  3.0373460e-06,  4.2346239e-01],
        [ 3.0887933e-02,  4.8177865e-01,  1.2358272e-01,  4.0691510e-02,
          6.5204567e-01,  8.9294370e-03,  2.5396826e-06,  3.4956399e-01],
        [ 3.5714988e-02,  5.5214357e-01,  1.4297988e-01,  4.6582468e-02,
          7.4612767e-01,  1.0243252e-02,  2.8991601e-06,  4.0091124e-01],
        [-2.1185249e-02, -3.2599619e-01, -8.4752105e-02, -2.7660128e-02,
         -4.3123451e-01, -6.0904021e-03, -1.7054832e-06, -2.3874629e-01],
        [ 3.0301003e-02,  4.6823439e-01,  1.2132694e-01,  3.9426312e-02,
          6.3276798e-01,  8.6825639e-03,  2.4

In [49]:
import innvestigate
import innvestigate.utils as iutils
from innvestigate.analyzer.base import AnalyzerBase

model = keras.models.load_model('model-30_noL1.h5', custom_objects={'metricF1': metricF1})
model_wo_softmax = innvestigate.model_wo_softmax(model)
Analyze = innvestigate.create_analyzer('lrp.alpha_1_beta_0', model_wo_softmax, input_layer_rule='Bounded')
sample = LRPDict_theta['LRP0']['samples_correct_neg_all'][0,wetpoints[0],wetpoints[1],wetpoints[2]][np.newaxis,:]

a = Analyze.analyze(sample)

In [50]:
model_wo_softmax.predict(LRPDict_theta['LRP0']['samples_correct_neg_all'][:,wetpoints[0],wetpoints[1],wetpoints[2]])

array([[ 2.3126523 , -2.3126526 ],
       [ 2.2912462 , -2.2912464 ],
       [ 2.25518   , -2.2551801 ],
       ...,
       [-0.89438224,  0.8943819 ],
       [-0.8900276 ,  0.8900272 ],
       [-0.8475955 ,  0.84759516]], dtype=float32)

In [51]:
model_wo_softmax.layers[-1].get_weights()[0], model_wo_softmax.layers[-1].get_weights()[1]

(array([[ 1.3074386e-07, -1.3150947e-07],
        [-1.3182325e+00,  1.3182346e+00],
        [ 5.7234749e-02, -5.7234574e-02],
        [ 4.4080198e-01, -4.4080061e-01],
        [ 1.1399362e+00, -1.1399329e+00],
        [ 1.2225859e-03, -1.2225783e-03],
        [ 3.8289103e-01, -3.8288978e-01],
        [ 3.9974806e-01, -3.9974669e-01]], dtype=float32),
 array([-0.11268532,  0.11268499], dtype=float32))

In [52]:
import innvestigate
import innvestigate.utils as iutils
from innvestigate.analyzer.base import AnalyzerBase

model = keras.models.load_model('model-60_noL1.h5', custom_objects={'metricF1': metricF1})
model_wo_softmax = innvestigate.model_wo_softmax(model)
Analyze = innvestigate.create_analyzer('lrp.alpha_1_beta_0', model_wo_softmax, input_layer_rule='Bounded')
sample = LRPDict_theta['LRP0']['samples_correct_neg_all'][0,wetpoints[0],wetpoints[1],wetpoints[2]][np.newaxis,:]

a = Analyze.analyze(sample)

In [53]:
model_wo_softmax.predict(LRPDict_theta['LRP0']['samples_correct_neg_all'][:,wetpoints[0],wetpoints[1],wetpoints[2]])

array([[ 2.2807636 , -2.2807639 ],
       [ 2.292386  , -2.2923863 ],
       [ 2.272514  , -2.2725143 ],
       ...,
       [-0.67361814,  0.67361784],
       [-0.8180562 ,  0.818056  ],
       [-0.8744602 ,  0.8744599 ]], dtype=float32)

In [54]:
model_wo_softmax.layers[-1].get_weights()[0], model_wo_softmax.layers[-1].get_weights()[1]

(array([[ 6.3270994e-04, -6.3270854e-04],
        [-9.0040725e-01,  9.0040857e-01],
        [ 5.8944121e-02, -5.8943935e-02],
        [ 8.3814627e-01, -8.3814359e-01],
        [ 7.6178503e-01, -7.6178271e-01],
        [-9.6884477e-01,  9.6884668e-01],
        [ 6.2415421e-01, -6.2415218e-01],
        [ 4.6086550e-01, -4.6086401e-01]], dtype=float32),
 array([-0.1766245 ,  0.17662422], dtype=float32))